# Crypto Data Fetcher

This Jupyter Notebook is designed to fetch and save cryptocurrency data using the Coinlore API. The notebook includes functions to:

1. Retrieve cryptocurrency data from the Coinlore API.
2. Save the retrieved data to a JSON file with a timestamped filename.
3. Interactively input the starting rank and the number of cryptocurrencies to fetch.

The main function orchestrates the workflow by prompting the user for input, fetching the data, and saving it to a file.

Here is the link to the Coinlore API documentation: https://www.coinlore.com/cryptocurrency-data-api

In [1]:
import requests
import datetime
import json

In [2]:
def get_crypto_data(start, limit):
    url = f"https://api.coinlore.net/api/tickers/?start={start}&limit={limit}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Erreur lors de la récupération des données.")
        return {}

In [3]:
def save_json_to_file(data):
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"resultat_{now}.json"

    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4)

    print(f"Le résultat a été enregistré dans {filename}")

In [ ]:
def main():
    try:
        start = int(input("A quel rang commencer ? (start) : "))
        limit = int(input("Combien de cryptos voulez-vous récupérer ? (limit) : "))

        data = get_crypto_data(start, limit)

        if data:
            save_json_to_file(data)
        else:
            print("Aucune donnée récupérée.")
    
    except ValueError:
        print("Veuillez entrer des nombres valides.")

if __name__ == "__main__":
    main()

Le résultat a été enregistré dans resultat_2025-03-12_17-40-28.json
